In [17]:
from state_of_the_art.tables.arxiv_paper_table import PaperTable


df_original =PaperTable().read()
df_original

,abstract_url,published,title,abstract,tdw_timestamp,tdw_uuid,pdf_url,institution
0,https://arxiv.org/abs/2408.03326,2024-08-06 17:59:44+00:00,LLaVA-OneVision: Easy Visual Task Transfer,"We present LLaVA-OneVision, a family of open l...",2024-08-07 08:20:13.367967,2f4446c6-4e30-4060-b880-ed140458bf17,None,None
0,https://arxiv.org/abs/2408.03325,2024-08-06 17:58:53+00:00,CoverBench: A Challenging Benchmark for Comple...,There is a growing line of research on verifyi...,2024-08-07 08:21:05.493254,561dfdc7-ee61-481b-a498-fd7fa4063dff,None,None
0,https://arxiv.org/abs/2408.03324,2024-08-06 17:58:38+00:00,Eddington Ratios of Dust-obscured Quasars at $...,Dust-obscured quasars have been suspected as t...,2024-08-07 08:21:14.278502,e9023128-827c-4d9c-ba76-da9feb21dff1,None,None
0,https://arxiv.org/abs/2408.03323,2024-08-06 17:58:29+00:00,ClassiFIM: An Unsupervised Method To Detect Ph...,Estimation of the Fisher Information Metric (F...,2024-08-07 08:20:28.776427,9be96790-5807-43e4-b73a-e1df0658c96b,None,None
0,https://arxiv.org/abs/2408.03322,2024-08-06 17:58:18+00:00,Segment Anything in Medical Images and Videos:...,Recent advances in segmentation foundation mod...,2024-08-07 08:20:59.174877,b5ec1289-6e86-4eb8-9c70-2dc71a44dc91,None,None
...,...,...,...,...,...,...,...,...
0,http://arxiv.org/abs/2410.06348,2024-10-08 20:37:13+00:00,Harnessing the Power of Noise: A Survey of Tec...,"Noise, traditionally considered a nuisance in ...",2024-10-10 08:05:17.579113,25dba93b-6d21-4f8f-9a26-9bce5df0d960,https://arxiv.org/pdf/2410.06348.pdf,None
0,http://arxiv.org/abs/2410.06323,2024-10-08 19:58:54+00:00,High Pressure Structural Behavior of Silicon T...,The high-pressure behavior of silicon tellurid...,2024-10-10 08:05:18.685212,98d8e9b0-3682-493a-9d2a-4036f968f9f3,https://arxiv.org/pdf/2410.06323.pdf,None
0,http://arxiv.org/abs/2410.06314,2024-10-08 19:45:53+00:00,Temporal Image Caption Retrieval Competition -...,"Multimodal models, which combine visual and te...",2024-10-10 08:05:20.079633,0010bc1b-db6d-4e10-ac41-56b1000399ed,https://arxiv.org/pdf/2410.06314.pdf,None
0,http://arxiv.org/abs/2410.06291,2024-10-08 18:48:02+00:00,Suppression of Generation-Recombination Noise ...,In this work we carry out low-frequency noise ...,2024-10-10 08:05:21.323173,428f07ff-8fd2-46ef-8224-365c914fbc82,https://arxiv.org/pdf/2410.06291.pdf,None


In [43]:
df_to_write = df_original.copy()
import pandas as pd
# write current date if null
df_to_write['published'] = df_to_write['published'].fillna(pd.Timestamp.now())

df_to_write['published_year'] = df_to_write['published'].apply(lambda x: x.year)
df_to_write['published_month'] = df_to_write['published'].apply(lambda x: x.month)

df_to_write.to_parquet("/tmp/arxiv_papers_partitioned.parquet", engine='pyarrow', partition_cols=["published_year", "published_month"])


In [52]:
from time import perf_counter
from time import sleep
from contextlib import contextmanager
from typing import Callable

@contextmanager
def catchtime() -> Callable[[], float]:
    t1 = t2 = perf_counter() 
    yield lambda: t2 - t1
    t2 = perf_counter() 

print(f"Time taken: {t() * 1000:.2f} ms")

Time taken: 1000.39 ms


In [64]:
import pandas as pd

with catchtime() as t:
    df = pd.read_parquet("/tmp/arxiv_papers_partitioned.parquet", dtype_backend='pyarrow')
    print(len(df.index))

print(f"Time taken: {t() * 1000:.2f} ms")

45785
Time taken: 141.76 ms


In [63]:

with catchtime() as t:
    df = pd.read_parquet("/tmp/arxiv_papers_partitioned.parquet", dtype_backend='pyarrow', filters=[("published_year", "=",  2024), ("published_month", "=", 10)])

    print(len(df.index))

print(f"Time taken: {t() * 1000:.2f} ms")

1233
Time taken: 24.27 ms


In [41]:
! rm -rf /tmp/arxiv_papers_partitioned.parquet
